# Topic model of Weibo Hot Search

# 1 Data collection

In [1]:
import datetime
def dateRange(start, end, step=1, format="%Y/%m/%d"):
    strptime, strftime = datetime.datetime.strptime, datetime.datetime.strftime
    days = (strptime(end, format) - strptime(start, format)).days
    return [strftime(strptime(start, format) + datetime.timedelta(i), format) for i in range(0, days, step)]

In [3]:
 if __name__ == '__main__':
    dates = dateRange("2017/05/09", "2019/10/29")

In [7]:
#将迭代的日期放入URL中，并逐个爬取网站，存入到dict中。
import requests
import pandas as pd
import json
headers = {"Accept-Language": "en-US, en;q=0.5"}
Keyword = []
URL = []
Count = []
SearchCount = []
Rank = []
Date = []
for i in dates:
    re = requests.get('http://www.enlightent.com/research/top/getWeiboHotSearchDayAggs.do?date='+i)
    re.encoding = 'utf-8'    
    text = re.text
    jsonobj = json.loads(text)
    #print(jsonobj)

    for item in jsonobj:
        keyword = item['keyword']
        Keyword.append(keyword)
        url = item['url']
        URL.append(url)

        count = item['count']
        Count.append(count)

        searchCount = item['searchCount']
        SearchCount.append(searchCount)

        rank = item['rank']
        Rank.append(rank)

        Date.append(i)

In [9]:
#将所有内容存入到csv文件中
import csv
datapage1 = {
    'Date': Date,
    'Keyword':Keyword,
    'URL':URL,
    'Count': Count,
    'SearchCount':SearchCount,
    'Rank': Rank,
}
weibo = pd.DataFrame.from_dict(datapage1, orient ='index').transpose()
#print(weibo.info())
weibo.to_csv('weibo_rank_2019_10_29.csv',index = False, encoding = 'utf_8_sig')

In [159]:
import pandas as pd

In [160]:
def date_clean(date):   
    #date = line[2].replace("/","-")
    date = date.split("/")
    if len(date[-2]) == 1:
        date[-2] = "0"+ date[1]
    if len(date[-1]) == 1:
        date[-1] = "0" + date[2]
    new_date = date[0]+"-"+date[1]+"-"+date[2]    
    return   new_date  

In [218]:
f = open('weibo_rank_2019_10_29.csv','r',encoding='utf-8')
next(f)
Year = []
Month = []
Date = []
Title = []
SearchCount = []
Highest_Rank = []
for line in f:
    line = line.strip().split(",")
    year = line[0]
    month = line[1]
    date = date_clean(line[2])   
    title = line[3]
    searchCount = line[5]
    rank = line[4]

    Year.append(year)
    Month.append(month)
    Date.append(date)
    Title.append(title)
    SearchCount.append(searchCount)
    Highest_Rank.append(rank)

In [219]:
df = pd.DataFrame({
    'Year':Year,
    'Month':Month,
    'Date':Date,
    'Title':Title,
    'SeachCount':SearchCount,
    'Highest_Rank':Highest_Rank})
df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True) # reset the index according to the number of the new data frame.
df.tail(10)

,Year,Month,Date,Title,SeachCount,Highest_Rank
215238,2019,10,2019-10-27,中国军团的军运时间,106604,3
215239,2019,10,2019-10-27,冬令时,101231,3
215240,2019,10,2019-10-27,宝蓝的莫甘娜,91356,1
215241,2019,10,2019-10-27,八一男排夺冠,89280,5
215242,2019,10,2019-10-27,FPX先下一城,86423,1
215243,2019,10,2019-10-27,你听到过最奇葩的呼噜声,83952,27
215244,2019,10,2019-10-27,惊蛰,74999,2
215245,2019,10,2019-10-27,主持人大赛,61331,33
215246,2019,10,2019-10-27,刘迦街舞,45981,27
215247,2019,10,2019-10-27,S9八强赛,27360,5


# 2 Data Cleaning

### reference:
stop words：
    百度停用词表|
    |哈工大停用词表
    |四川大学机器智能实验试停用词库
    |中文停用词表

In [163]:
import jieba
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath,'r',encoding='utf-8').readlines()]
    return stopwords

def cut(Title):
    Title_cut = jieba.cut(Title)
    stopwords = stopwordslist("STOPWORDS.txt")
    New_Title_cut = []
    for word in Title_cut:
        if word not in stopwords:
            New_Title_cut.append(word)
    New_Title_cut = str(New_Title_cut).replace("[","").replace("]","").replace(","," ").replace("'","")
    return New_Title_cut

In [164]:
df['cutted_title'] = df['Title'].apply(cut)
# one new column was generated
df.tail(10)

,Year,Month,Date,Title,SeachCount,Highest_Rank,cutted_title
175051,2019,5,2019-05-09,公交车司机的工位,97541,4,公交车 司机 工位
175052,2019,5,2019-05-09,和平精英今日公测,87891,1,和平 精英 今日 公测
175053,2019,5,2019-05-09,特朗普制裁伊朗钢铁业和矿业,85176,2,特朗普 制裁 伊朗 钢铁业 矿业
175054,2019,5,2019-05-09,上海又一知名私募失联,28599,1,上海 知名 私募 失联
175055,2019,5,2019-05-09,关税,26802,1,关税
175056,2019,5,2019-05-09,盖茨妻子谈男女家务不平等,14926,16,盖茨 妻子 谈 男女 家务 平等
175057,2019,5,2019-05-09,费莱尼绝杀,14614,10,费 莱尼 绝杀
175058,2019,5,2019-05-09,科学家用AI控制猴子大脑,7866,1,科学家 AI 控制 猴子 大脑
175059,2019,5,2019-05-09,韩国整形趋势女爱豆,6272,1,韩国 整形 趋势 女爱豆
175060,2019,5,2019-05-09,俄航副驾驶为救人重回起火客机,6253,1,俄航 副驾驶 救人 重回 起火 客机


# 3 Topic model

#https://www.programcreek.com/python/example/83254/sklearn.feature_extraction.text.TfidfVectorizer

In [165]:
import pyLDAvis
import pyLDAvis.sklearn
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [166]:
def extract_topic(start, end, num_topics, num_keywords):  # the function in outer layer
    def periodize(start,end):                             # the function in inner layer
        in_point=df[df['Date']==start].index.tolist()[0] # find the first index of small date
        out_point=df[df['Date']==end].index.tolist()[-1]   # find the last index of big date
        return df['cutted_title'][in_point:out_point]
    # ↓ vectorizer is a local function, which could vectorize the text
    vectorizer = CountVectorizer(max_features=3000,stop_words='english',min_df=0.001,max_df = 0.5)
    tf = vectorizer.fit_transform(periodize(start,end)) # apply the vectorize function to the text
    feature_names=vectorizer.get_feature_names() # features are numbers, while feature_names are names
    # ↓ lda is a local function, which could select the topic
    lda = LatentDirichletAllocation(n_components=num_topics, max_iter=100,random_state=0,
                                    learning_method='online',learning_offset=50.).fit(tf)
                                        # apply the vectorized text (i.e. tf) to lda↑    
    for index, topic in enumerate(lda.components_): # topic is the number of features
        Month = start.split("-")[0]+ "年"+ start.split("-")[1]+"月"
        print(Month,'#%d:' % (index+1),
              ' '.join([feature_names[i] for i in topic.argsort()[:-num_keywords - 1:-1]]),'\n')
# argsort() is a function which can generate a new index ↑
# based on the existing list according to the size of the value of the elements.

In [170]:
extract_topic('2018-01-01','2018-01-01',5,10)
extract_topic('2018-01-02','2018-01-02',5,10)
extract_topic('2018-01-03','2018-01-03',5,10)
extract_topic('2018-01-04','2018-01-04',5,10)
extract_topic('2018-01-05','2018-01-05',5,10)
extract_topic('2018-01-06','2018-01-06',5,10)
extract_topic('2018-01-07','2018-01-07',5,10)
extract_topic('2018-01-08','2018-01-08',5,10)
extract_topic('2018-01-09','2018-01-09',5,10)
extract_topic('2018-01-10','2018-01-10',5,10)

2018年01月 #1: 2018 喜欢 角色 扮演 2017 新年 杨幂 万元 转给 路费 

2018年01月 #2: 跨年 新年 2017 吴昕 恋爱 李宇春 伦敦 升旗仪式 特朗普 体面 

2018年01月 #3: 中国 年度 窦靖童 前任 巨星 演技 朴树 tt 苍井空 冻得 

2018年01月 #4: 李易峰 烟花 gai 元旦 千玺 回应 加价 手术 患者 18000 

2018年01月 #5: 假唱 陈伟霆 照片 日本 男子 成年 歌谣 祭典 追光 普京 

2018年01月 #1: 2017 司机 美食 元旦 小伙 城市 法国 动车 降落 访华 

2018年01月 #2: 回应 死亡 伊朗 示威 普京 10 后背 主题 按摩 餐厅 

2018年01月 #3: 钱包 房企 彩绘 税改 百亿 政策 套房 羊肉 销售 高校学生 

2018年01月 #4: 新年 攻略 比特 2018 烟花 丈夫 演技 城市 企鹅 一只 

2018年01月 #5: 分钟 地铁 幸福 世界 小朋友 国旗 天安门 哨位 目标 模拟 

2018年01月 #1: 新年 收购 写成 67 献词 擒拿 豆腐 证券报 检讨书 火腿 

2018年01月 #2: 退役 巴尔德斯 中国 瑞士 猪肉 产品 马拉喀什 出口 摩洛哥 最高人民法院 

2018年01月 #3: 春运 王思聪 预报地图 火车票 堆雪人 将进酒 全国 医药 按摩 归宿 

2018年01月 #4: 世界各地 烟花 找人斗舞 轻易 跨年 魔方 豆沙 冰块 小丸子 草莓酱 

2018年01月 #5: 小时 精髓 经济学 黄毅 攻略 公交 昏迷 人民 世界各地 苹果 

2018年01月 #1: 跨年 走红 冬日 白菜 台历 神曲 纸屑 乐天 这组 洗脑 

2018年01月 #2: 老师 女友 男子 朗诵 驾弃 古诗文 逃跑 走红 江苏 地震 

2018年01月 #3: 2018 雪乡 比特 投家 币大涨 泰国 宰客 遭殃 采访 买入 

2018年01月 #4: 小哥 国外 穆雷 超火 因伤 退出 尊巴 现场表演 健身 六方会谈 

2018年01月 #5: 下雪 暴雪 预警 分钟 车祸 2017 消防官兵 这件 酷似 路由器 

2018年01月 #1: 裸训 鹰爸 南京 中国 孩子 13 美国 1

In [200]:
extract_topic('2018-01-01','2018-01-27',1,10)

2018年01月 #1: 中国 回应 2018 恋爱 歌手 10 手机 2017 儿子 女儿 



In [203]:
extract_topic('2018-02-03','2018-02-27',1,10)

2018年02月 #1: 春晚 中国 春节 过年 女儿 妈妈 春运 地震 回家 10 



In [217]:
extract_topic('2018-03-01','2018-03-31',1,10)

2018年03月 #1: 中国 男孩 回应 女儿 偶遇 女生 女孩 妈妈 老人 周杰伦 



In [178]:
extract_topic('2018-04-01','2018-04-30',1,10)

2018年04月 #1: 中国 回应 手机 鹿晗 女儿 徐坤 北京 周杰伦 偶遇 千玺 



In [179]:
extract_topic('2018-05-01','2018-05-31',1,10)

2018年05月 #1: 妈妈 中国 回应 温暖 徐坤 鹿晗 詹姆斯 演唱会 儿子 戛纳 



In [180]:
extract_topic('2018-06-01','2018-06-30',1,10)

2018年06月 #1: 世界杯 高考 回应 中国 101 德国 球迷 毕业 阿根廷 直播 



In [182]:
extract_topic('2018-07-01','2018-07-31',1,10)

2018年07月 #1: 回应 世界杯 中国 扶摇 朱一龙 疫苗 法国 演唱会 直播 白宇 



In [188]:
extract_topic('2018-08-01','2018-08-31',1,10)

2018年08月 #1: 回应 滴滴 中国 攻略 延禧 香蜜 沉沉 亚运会 如霜 七夕 



In [184]:
extract_topic('2018-09-01','2018-09-30',1,10)

2018年09月 #1: 回应 中国 台风 中秋 开学 滴滴 道歉 如懿传 山竹 iphone 



In [185]:
extract_topic('2018-10-01','2018-10-31',1,10)

2018年10月 #1: 回应 中国 坠江 唐嫣 rng 李咏 婚礼 国庆 游客 结婚 



In [186]:
extract_topic('2018-11-01','2018-11-30',1,10)

2018年11月 #1: 回应 王思聪 粉丝 中国 11 蒋劲夫 道歉 ig 快递 双十 



In [187]:
extract_topic('2018-12-01','2018-12-31',1,10)

2018年12月 #1: 回应 2018 中国 否认 考研 海王 结婚 圣诞 男孩 粉丝 

